<a href="https://colab.research.google.com/github/yohanesnuwara/computational-geophysics/blob/master/seismic_displayer_ipywidget.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
!pip install segyio
import segyio

     |████████████████████████████████| 92kB 5.0MB/s 


In [ ]:
# specify your path
norne = '/content/computational-geophysics/norne/norne4d_2006-full-mini.sgy'
volve = '/content/drive/My Drive/Volve Dataset/ST0202R08_PZ_PSDM_FULL_OFFSET_DEPTH.MIG_FIN.POST_STACK.3D.JS-017534.segy'
dutch = '/content/drive/My Drive/Public geoscience Data/Dutch F3 seismic data/Dutch Government_F3_entire_8bit seismic.segy'

# choose which dataset to display
filename = norne

In [ ]:
# read seismic data with segyio
with segyio.open(filename) as f:

  data = segyio.tools.cube(f)
  inline_data = f.iline
  crossline_data = f.xline

  inlines = f.ilines
  crosslines = f.xlines
  twt = f.samples
  sample_rate = segyio.tools.dt(f) / 1000
  print('Inline range from', inlines[0], 'to', inlines[-1])
  print('Crossline range from', crosslines[0], 'to', crosslines[-1])
  print('TWT from', twt[0], 'to', twt[-1])   
  print('Sample rate:', sample_rate, 'ms')

  clip_percentile = 99
  vm = np.percentile(data, clip_percentile)

f'The {clip_percentile}th percentile is {vm:.0f}; the max amplitude is {data.max():.0f}'

Inline range from 1000 to 1200
Crossline range from 2000 to 2300
TWT from 0.0 to 4000.0
Sample rate: 4.0 ms


'The 99th percentile is 2330; the max amplitude is 22711'

In [ ]:
"""
Interactive seismic viewer

Input:

data, inlines, crosslines, twt, sample_rate, vm: output from segyio.read
data: the whole cube (3D numpy array)
inlines: inline locations (1D numpy array)
crosslines: crossline locations (1D numpy array)
twt: two-way travel time (1D numpy array)
sample_rate: sampling rate (float)
vm: 99th percentile of data

"""

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual, ToggleButtons
import ipywidgets as widgets
import segyio

type = ToggleButtons(description='Selection',options=['Inline','Crossline',
                                                      'Timeslice'])
cmap_button = ToggleButtons(description='Colormaps',options=['gray','seismic',
                                                             'RdBu','PuOr'])

@interact

def seis_widget(type=type, inline_loc=(min(inlines), max(inlines), 5), 
                xline_loc=(min(crosslines), max(crosslines), 5),
                timeslice_loc=(min(twt), max(twt), sample_rate), 
                cmap=cmap_button):

  if type == 'Inline':

    with segyio.open(filename) as f: 
        inline_slice = f.iline[inline_loc]  

        plt.figure(figsize=(20, 10))
        plt.title('Norne Seismic at Inline {}'.format(inline_loc))
        extent = [crosslines[0], crosslines[-1], twt[-1], twt[0]]

        p1 = plt.imshow(inline_slice.T, cmap=cmap, aspect='auto', extent=extent,
                        vmin=-vm, vmax=vm, interpolation='bicubic')

        plt.xlabel('Crossline'); plt.ylabel('TWT')
        plt.show()

  if type == 'Crossline':

    with segyio.open(filename) as f:
        xline_slice = f.xline[xline_loc]  

        plt.figure(figsize=(20, 10))
        plt.title('Norne Seismic at Crossline {}'.format(xline_loc))
        extent = [inlines[0], inlines[-1], twt[-1], twt[0]]

        p1 = plt.imshow(xline_slice.T, cmap=cmap, aspect='auto', extent=extent, 
                        vmin=-vm, vmax=vm, interpolation='bicubic')

        plt.xlabel('Inline'); plt.ylabel('TWT')
        plt.show()
  
  if type == 'Timeslice':

    id = np.where(twt == timeslice_loc)[0][0]
    tslice = data[:,:,id]

    plt.figure(figsize=(7,10))
    plt.title('Norne Seismic at Timeslice {} ms'.format(timeslice_loc))
    extent = [inlines[0], inlines[-1], crosslines[-1], crosslines[0]]

    p1 = plt.imshow(tslice.T, cmap=cmap, aspect='auto', extent=extent, 
                    vmin=-vm, vmax=vm, interpolation='bicubic')


    plt.xlabel('Inline'); plt.ylabel('Crossline')
    plt.gca().xaxis.set_ticks_position('top') # axis on top
    plt.gca().xaxis.set_label_position('top') # label on top
    plt.xlim(min(inlines), max(inlines))
    plt.ylim(min(crosslines), max(crosslines))
    plt.axis('equal')
    plt.show()   

interactive(children=(ToggleButtons(description='Selection', options=('Inline', 'Crossline', 'Timeslice'), val…